In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import xgboost as xgb

In [3]:
# 1. Data Processing
# 1.1 Extract data from October, November, and December
file_path = '新竹_2021.xls'
data = pd.read_excel(file_path)
data = data.rename(columns={'測站                  ': '測站', '日期                  ': '日期', '測項                  ': '測項'})
data['日期'] = pd.to_datetime(data['日期'], format='%Y/%m/%d %H:%M', errors='coerce')
data_filter = data[data['日期'].dt.month.isin([10, 11, 12])]
data_filter

,測站,日期,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
4915,新竹,2021-10-01,AMB_TEMP,28.3,28.3,27.8,27.8,27.6,27.6,27.7,...,31.6,31.4,30.9,30.5,30.2,29.8,29.4,29.1,28.7,28.2
4916,新竹,2021-10-01,CH4,2.04,2.02,2.12,2.18,2.19,2.24,2.21,...,1.96,1.97,2.01,2.06,2.07,2.05,2.04,2.03,2.08,2.08
4917,新竹,2021-10-01,CO,0.34,0.3,0.3,0.29,0.3,0.33,0.44,...,0.25,0.27,0.32,0.43,0.45,0.45,0.43,0.42,0.43,0.39
4918,新竹,2021-10-01,NMHC,0.17,0.13,0.12,0.14,0.17,0.16,0.18,...,0.04,0.06,0.05,0.17,0.24,0.22,0.16,0.14,0.16,0.14
4919,新竹,2021-10-01,NO,0.9,0.2,0.5,0.4,0.2,0.6,2.2,...,0.5,0.5,0.5,0.3,0.3,0.3,0.3,0.3,0.4,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6566,新竹,2021-12-31,THC,2.24,2.22,2.19,2.17,2.15,2.1,2.1,...,2.07,2.05,2.09,2.1,2.05,2.1,2.15,2.13,2.09,2.05
6567,新竹,2021-12-31,WD_HR,38,51,50,47,53,53,46,...,51,54,48,53,54,53,47,37,42,48
6568,新竹,2021-12-31,WIND_DIREC,37,59,37,50,62,42,41,...,66,45,40,59,57,55,41,36,53,39
6569,新竹,2021-12-31,WIND_SPEED,2.6,2.6,2.3,2.4,3.4,3.2,3.1,...,4.8,3.2,2.8,3.2,2.5,2.2,1.7,2.5,2.3,1.9


In [4]:
# 1.2 Replace missing and invalid values with the average of the previous and next hour values
# (If the previous hour still has a missing value, take the value from an even earlier hour)
data_filter = data_filter.reset_index(drop=True)
data_filter.columns = data_filter.columns.astype(str)

def fill_with_neighbors_mean(series):
    series = series.copy()
    for i in range(len(series)):
        if pd.isna(series.iloc[i]) or series.iloc[i] == '#                              ' or series.iloc[i] == 'A                              ' or series.iloc[i] == 'x                              ' or series.iloc[i] == '*                              ':
            prev_index = i - 1
            next_index = i + 1

            while prev_index >= 0 and (pd.isna(series.iloc[prev_index]) or series.iloc[prev_index] == '#                              ' or series.iloc[prev_index] == 'A                              ' or series.iloc[prev_index] == 'x                              ' or series.iloc[prev_index] == '*                              '):
                prev_index -= 1

            while next_index < len(series) and (pd.isna(series.iloc[next_index]) or series.iloc[next_index] == '#                              ' or series.iloc[next_index] == 'A                              ' or series.iloc[next_index] == 'x                              ' or series.iloc[next_index] == '*                              '):
                next_index += 1
                
            if prev_index >= 0 and next_index < len(series):
                series.iloc[i] = (float(series.iloc[prev_index]) + float(series.iloc[next_index])) / 2
            
            elif prev_index >= 0:
                series.iloc[i] = float(series.iloc[prev_index])
                
            elif next_index < len(series):
                series.iloc[i] = float(series.iloc[next_index])
                
    return series
    
for i in range(1656):
    data_filter.iloc[i, 3:] = fill_with_neighbors_mean(data_filter.iloc[i, 3:])

data_filter

,測站,日期,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,新竹,2021-10-01,AMB_TEMP,28.3,28.3,27.8,27.8,27.6,27.6,27.7,...,31.6,31.4,30.9,30.5,30.2,29.8,29.4,29.1,28.7,28.2
1,新竹,2021-10-01,CH4,2.04,2.02,2.12,2.18,2.19,2.24,2.21,...,1.96,1.97,2.01,2.06,2.07,2.05,2.04,2.03,2.08,2.08
2,新竹,2021-10-01,CO,0.34,0.3,0.3,0.29,0.3,0.33,0.44,...,0.25,0.27,0.32,0.43,0.45,0.45,0.43,0.42,0.43,0.39
3,新竹,2021-10-01,NMHC,0.17,0.13,0.12,0.14,0.17,0.16,0.18,...,0.04,0.06,0.05,0.17,0.24,0.22,0.16,0.14,0.16,0.14
4,新竹,2021-10-01,NO,0.9,0.2,0.5,0.4,0.2,0.6,2.2,...,0.5,0.5,0.5,0.3,0.3,0.3,0.3,0.3,0.4,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,新竹,2021-12-31,THC,2.24,2.22,2.19,2.17,2.15,2.1,2.1,...,2.07,2.05,2.09,2.1,2.05,2.1,2.15,2.13,2.09,2.05
1652,新竹,2021-12-31,WD_HR,38,51,50,47,53,53,46,...,51,54,48,53,54,53,47,37,42,48
1653,新竹,2021-12-31,WIND_DIREC,37,59,37,50,62,42,41,...,66,45,40,59,57,55,41,36,53,39
1654,新竹,2021-12-31,WIND_SPEED,2.6,2.6,2.3,2.4,3.4,3.2,3.1,...,4.8,3.2,2.8,3.2,2.5,2.2,1.7,2.5,2.3,1.9


In [6]:
# 10/27, 11/8 有無效值 A
# 12/29 有無效值 # 9, 10, 11, 12, 13
data_filter.columns = data_filter.columns.astype(str)
dec_29_data = data_filter[data_filter['日期'] == '2021-12-29'].loc[:, ['日期', '測站', '測項', '9', '10', '11', '12']]
print(dec_29_data)

             日期                    測站                    測項     9    10  \
1602 2021-12-29  新竹                    AMB_TEMP              18.7  20.1   
1603 2021-12-29  新竹                    CH4                   2.05  2.04   
1604 2021-12-29  新竹                    CO                    0.41  0.35   
1605 2021-12-29  新竹                    NMHC                  0.18  0.11   
1606 2021-12-29  新竹                    NO                     6.2   5.4   
1607 2021-12-29  新竹                    NO2                     17  14.3   
1608 2021-12-29  新竹                    NOx                   23.2  19.7   
1609 2021-12-29  新竹                    O3                    16.9  23.9   
1610 2021-12-29  新竹                    PM10                    28    28   
1611 2021-12-29  新竹                    PM2.5                   18    19   
1612 2021-12-29  新竹                    RAINFALL                 0     0   
1613 2021-12-29  新竹                    RH                      85    78   
1614 2021-12-29  新竹      

In [7]:
# 1.3 "NO" indicates no rainfall, replaced with 0
data_filter['測項'] = data_filter['測項'].replace('NO                  ', 0)
data_filter

,測站,日期,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,新竹,2021-10-01,AMB_TEMP,28.3,28.3,27.8,27.8,27.6,27.6,27.7,...,31.6,31.4,30.9,30.5,30.2,29.8,29.4,29.1,28.7,28.2
1,新竹,2021-10-01,CH4,2.04,2.02,2.12,2.18,2.19,2.24,2.21,...,1.96,1.97,2.01,2.06,2.07,2.05,2.04,2.03,2.08,2.08
2,新竹,2021-10-01,CO,0.34,0.3,0.3,0.29,0.3,0.33,0.44,...,0.25,0.27,0.32,0.43,0.45,0.45,0.43,0.42,0.43,0.39
3,新竹,2021-10-01,NMHC,0.17,0.13,0.12,0.14,0.17,0.16,0.18,...,0.04,0.06,0.05,0.17,0.24,0.22,0.16,0.14,0.16,0.14
4,新竹,2021-10-01,0,0.9,0.2,0.5,0.4,0.2,0.6,2.2,...,0.5,0.5,0.5,0.3,0.3,0.3,0.3,0.3,0.4,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,新竹,2021-12-31,THC,2.24,2.22,2.19,2.17,2.15,2.1,2.1,...,2.07,2.05,2.09,2.1,2.05,2.1,2.15,2.13,2.09,2.05
1652,新竹,2021-12-31,WD_HR,38,51,50,47,53,53,46,...,51,54,48,53,54,53,47,37,42,48
1653,新竹,2021-12-31,WIND_DIREC,37,59,37,50,62,42,41,...,66,45,40,59,57,55,41,36,53,39
1654,新竹,2021-12-31,WIND_SPEED,2.6,2.6,2.3,2.4,3.4,3.2,3.1,...,4.8,3.2,2.8,3.2,2.5,2.2,1.7,2.5,2.3,1.9


In [8]:
#  1.4 Split the data into a training set (October and November) and a test set (December)
train_set = data_filter[data_filter['日期'].dt.month.isin([10, 11])]
test_set = data_filter[data_filter['日期'].dt.month == 12]
print("training set (October and November):")
print(train_set)
print("\ntest set (December):")
print(test_set)

training set (October and November):
                        測站         日期                    測項     0     1     2  \
0     新竹                   2021-10-01  AMB_TEMP              28.3  28.3  27.8   
1     新竹                   2021-10-01  CH4                   2.04  2.02  2.12   
2     新竹                   2021-10-01  CO                    0.34   0.3   0.3   
3     新竹                   2021-10-01  NMHC                  0.17  0.13  0.12   
4     新竹                   2021-10-01                     0   0.9   0.2   0.5   
...                    ...        ...                   ...   ...   ...   ...   
1093  新竹                   2021-11-30  THC                   2.53  2.65   2.4   
1094  新竹                   2021-11-30  WD_HR                   99   146   158   
1095  新竹                   2021-11-30  WIND_DIREC             128   249   150   
1096  新竹                   2021-11-30  WIND_SPEED             0.3   0.4   0.6   
1097  新竹                   2021-11-30  WS_HR                  0.3   0.2 

In [9]:
# 1.5 Create time series data: Transform the data so that rows represent 18 different attributes and columns represent hourly data
# **hint: Merge every 18 rows of the training set into a single unit, transforming it into a DataFrame with dimensions (18, 61*24).
# ***hint: Each attribute will have 61 days * 24 hours, resulting in 1464 data points.
attribute_data = {}

for attribute in train_set['測項'].unique():
    attribute_df = train_set[train_set['測項'] == attribute].sort_values('日期')
    hourly_data = attribute_df.loc[:, '0':'23'].values.flatten()
    
    if len(hourly_data) == 61 * 24:
        attribute_data[attribute] = hourly_data

time_series_df = pd.DataFrame(attribute_data).T
time_series_df.columns = train_set.loc[:, '0':'23'].columns.tolist() * 61

print("The transformed time series data: ")
print(time_series_df)
print("data dimensions: ", time_series_df.shape)

The transformed time series data: 
                         0     1     2     3     4     5     6     7     8  \
AMB_TEMP              28.3  28.3  27.8  27.8  27.6  27.6  27.7  28.4    30   
CH4                   2.04  2.02  2.12  2.18  2.19  2.24  2.21  2.17  2.13   
CO                    0.34   0.3   0.3  0.29   0.3  0.33  0.44  0.62   0.6   
NMHC                  0.17  0.13  0.12  0.14  0.17  0.16  0.18  0.23   0.2   
0                      0.9   0.2   0.5   0.4   0.2   0.6   2.2   3.6   2.7   
NO2                   18.8  11.9  15.1  12.8  14.9  17.5    20  22.1  18.1   
NOx                   19.8  12.2  15.6  13.2  15.1  18.2  22.3  25.7  20.8   
O3                      16  21.5  16.9  16.4  12.6  11.3    13  15.2  32.7   
PM10                    28    24    29    32    31    32    46    48    58   
PM2.5                   28    22    26    24    28    20    31    37    38   
RAINFALL                 0     0     0     0     0     0     0     0     0   
RH                      71   

In [10]:
# 2. time series
# 2.1 prediction target

# take the next 1-hour data as the prediction target
data_split = time_series_df
X_next = []
Y_next = []

for i in range(data_split.shape[1] - 6):
    X_next.append(data_split.iloc[:, i:i + 6].values)
    Y_next.append(data_split.iloc[:, i + 6].values)

X_next = np.array(X_next)
Y_next = np.array(Y_next)

print("X_next shape: ", X_next.shape)
print("X_next: ", X_next)
print("\nY_next shape: ", Y_next.shape)
print("Y_next: ", Y_next)

X_next shape:  (1458, 18, 6)
X_next:  [[[28.3 28.3 27.8 27.8 27.6 27.6]
  [2.04 2.02 2.12 2.18 2.19 2.24]
  [0.34 0.3 0.3 0.29 0.3 0.33]
  ...
  [33 183 160 151 90 151]
  [0.8 1.2 1.1 0.6 0.9 0.5]
  [0.7 0.6 0.6 0.4 0.5 0.4]]

 [[28.3 27.8 27.8 27.6 27.6 27.7]
  [2.02 2.12 2.18 2.19 2.24 2.21]
  [0.3 0.3 0.29 0.3 0.33 0.44]
  ...
  [183 160 151 90 151 212]
  [1.2 1.1 0.6 0.9 0.5 0.8]
  [0.6 0.6 0.4 0.5 0.4 0.4]]

 [[27.8 27.8 27.6 27.6 27.7 28.4]
  [2.12 2.18 2.19 2.24 2.21 2.17]
  [0.3 0.29 0.3 0.33 0.44 0.62]
  ...
  [160 151 90 151 212 239]
  [1.1 0.6 0.9 0.5 0.8 1.5]
  [0.6 0.4 0.5 0.4 0.4 0.4]]

 ...

 [[23.2 21.7 20.8 20.5 20.3 19.9]
  [1.98 2.01 2.08 2.09 2.1 2.1]
  [0.22 0.31 0.44 0.46 0.46 0.45]
  ...
  [55 60 59 52 57 54]
  [3.6 3.2 3.6 2.9 3.8 3.8]
  [2.8 2.9 3.1 2.9 3.4 3.5]]

 [[21.7 20.8 20.5 20.3 19.9 19.4]
  [2.01 2.08 2.09 2.1 2.1 2.09]
  [0.31 0.44 0.46 0.46 0.45 0.42]
  ...
  [60 59 52 57 54 35]
  [3.2 3.6 2.9 3.8 3.8 4]
  [2.9 3.1 2.9 3.4 3.5 3.6]]

 [[20.8 20.5 20.

In [11]:
# take the next 6-hour data as the prediction target
X_six = []
Y_six = []

for i in range(data_split.shape[1] - 11):
    X_six.append(data_split.iloc[:, i:i + 6].values)
    Y_six.append(data_split.iloc[:, i + 11].values)

X_six = np.array(X_six)
Y_six = np.array(Y_six)

print("X_six shape: ", X_six.shape)
print("X_six: ", X_six)
print("\nY_six shape: ", Y_six.shape)
print("Y_six: ", Y_six)

X_six shape:  (1453, 18, 6)
X_six:  [[[28.3 28.3 27.8 27.8 27.6 27.6]
  [2.04 2.02 2.12 2.18 2.19 2.24]
  [0.34 0.3 0.3 0.29 0.3 0.33]
  ...
  [33 183 160 151 90 151]
  [0.8 1.2 1.1 0.6 0.9 0.5]
  [0.7 0.6 0.6 0.4 0.5 0.4]]

 [[28.3 27.8 27.8 27.6 27.6 27.7]
  [2.02 2.12 2.18 2.19 2.24 2.21]
  [0.3 0.3 0.29 0.3 0.33 0.44]
  ...
  [183 160 151 90 151 212]
  [1.2 1.1 0.6 0.9 0.5 0.8]
  [0.6 0.6 0.4 0.5 0.4 0.4]]

 [[27.8 27.8 27.6 27.6 27.7 28.4]
  [2.12 2.18 2.19 2.24 2.21 2.17]
  [0.3 0.29 0.3 0.33 0.44 0.62]
  ...
  [160 151 90 151 212 239]
  [1.1 0.6 0.9 0.5 0.8 1.5]
  [0.6 0.4 0.5 0.4 0.4 0.4]]

 ...

 [[22.9 22.8 23.1 23.2 23.6 23.2]
  [1.96 1.96 1.99 1.97 1.98 1.98]
  [0.22 0.22 0.25 0.2 0.21 0.22]
  ...
  [66 47 42 57 55 55]
  [1.4 2.6 2.6 3 2.7 3.6]
  [1.2 1.9 1.8 2.7 2.7 2.8]]

 [[22.8 23.1 23.2 23.6 23.2 21.7]
  [1.96 1.99 1.97 1.98 1.98 2.01]
  [0.22 0.25 0.2 0.21 0.22 0.31]
  ...
  [47 42 57 55 55 60]
  [2.6 2.6 3 2.7 3.6 3.2]
  [1.9 1.8 2.7 2.7 2.8 2.9]]

 [[23.1 23.2 23.6 

In [12]:
# 2.2 Take X separately

# only PM2.5
pm25_data = time_series_df.loc['PM2.5               '].values

X_pm25_next = []
Y_pm25_next = []
X_pm25_six = []
Y_pm25_six = []

for i in range(len(pm25_data) - 6):
    X_pm25_next.append(pm25_data[i:i + 6])
    Y_pm25_next.append(pm25_data[i + 6])
for i in range(len(pm25_data) - 11):
    X_pm25_six.append(pm25_data[i:i + 6])
    Y_pm25_six.append(pm25_data[i + 11])

X_pm25_next = np.array(X_pm25_next)
Y_pm25_next = np.array(Y_pm25_next)
X_pm25_six = np.array(X_pm25_six)
Y_pm25_six = np.array(Y_pm25_six)

print("X next 1-hour (only PM2.5): ", X_pm25_next)
print("\nX next 6-hours (only PM2.5): ", X_pm25_six)

# all 18 attributes
print("\nX next 1-hour (all 18 attributes): ", X_next)
print("\nX next 6-hours (all 18 attributes): ", X_six)

X next 1-hour (only PM2.5):  [[28 22 26 24 28 20]
 [22 26 24 28 20 31]
 [26 24 28 20 31 37]
 ...
 [11 17 34 34 36 34]
 [17 34 34 36 34 32]
 [34 34 36 34 32 32]]

X next 6-hours (only PM2.5):  [[28 22 26 24 28 20]
 [22 26 24 28 20 31]
 [26 24 28 20 31 37]
 ...
 [10 11 9 10 9 11]
 [11 9 10 9 11 17]
 [9 10 9 11 17 34]]

X next 1-hour (all 18 attributes):  [[[28.3 28.3 27.8 27.8 27.6 27.6]
  [2.04 2.02 2.12 2.18 2.19 2.24]
  [0.34 0.3 0.3 0.29 0.3 0.33]
  ...
  [33 183 160 151 90 151]
  [0.8 1.2 1.1 0.6 0.9 0.5]
  [0.7 0.6 0.6 0.4 0.5 0.4]]

 [[28.3 27.8 27.8 27.6 27.6 27.7]
  [2.02 2.12 2.18 2.19 2.24 2.21]
  [0.3 0.3 0.29 0.3 0.33 0.44]
  ...
  [183 160 151 90 151 212]
  [1.2 1.1 0.6 0.9 0.5 0.8]
  [0.6 0.6 0.4 0.5 0.4 0.4]]

 [[27.8 27.8 27.6 27.6 27.7 28.4]
  [2.12 2.18 2.19 2.24 2.21 2.17]
  [0.3 0.29 0.3 0.33 0.44 0.62]
  ...
  [160 151 90 151 212 239]
  [1.1 0.6 0.9 0.5 0.8 1.5]
  [0.6 0.4 0.5 0.4 0.4 0.4]]

 ...

 [[23.2 21.7 20.8 20.5 20.3 19.9]
  [1.98 2.01 2.08 2.09 2.1 2.1]
  [

In [13]:
# 2.3 use Linear Regression and XGBoost

# take the next 1-hour data
# Linear Regression
X_pm25_next_reshaped = X_pm25_next.reshape(X_pm25_next.shape[0], -1)
X_train_next, X_test_next, Y_train_next, Y_test_next = train_test_split(X_pm25_next_reshaped, Y_pm25_next, test_size=0.8, random_state=42)

print("Take the Next 1-hour Data")
print("\n--- Linear Regression Model ---")
lr_model_next = LinearRegression()
lr_model_next.fit(X_train_next, Y_train_next)

y_pred_lr_next = lr_model_next.predict(X_test_next)

mse_lr_next = mean_squared_error(Y_test_next, y_pred_lr_next)
r2_lr_next = r2_score(Y_test_next, y_pred_lr_next)
print(f"Mean Squared Error: {mse_lr_next:.2f}")
print(f"R2 Score: {r2_lr_next:.2f}")

# XGBoost
print("\n--- XGBoost Model ---")
xgb_model_next = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42)
xgb_model_next.fit(X_train_next, Y_train_next)

y_pred_xgb_next = xgb_model_next.predict(X_test_next)

mse_xgb_next = mean_squared_error(Y_test_next, y_pred_xgb_next)
r2_xgb_next = r2_score(Y_test_next, y_pred_xgb_next)
print(f"Mean Squared Error: {mse_xgb_next:.2f}")
print(f"R2 Score: {r2_xgb_next:.2f}")

Take the Next 1-hour Data

--- Linear Regression Model ---
Mean Squared Error: 13.96
R2 Score: 0.68

--- XGBoost Model ---
Mean Squared Error: 17.14
R2 Score: 0.61


In [14]:
# Output the result as a .csv file
results = pd.DataFrame({
    'No': range(len(Y_test_next.flatten())),
    'PM2.5': y_pred_lr_next.flatten()
})
results.to_csv('submission_next.csv', index=False)

In [15]:
# take the next 6-hours data
# Linear Regression
X_six_reshaped = X_six.reshape(X_six.shape[0], -1)
X_train_six, X_test_six, Y_train_six, Y_test_six = train_test_split(X_six_reshaped, Y_six, test_size=0.2, random_state=42)

print("Take the Next 6-hours Data")
print("\n--- Linear Regression Model ---")
lr_model_six = LinearRegression()
lr_model_six.fit(X_train_six, Y_train_six)

y_pred_lr_six = lr_model_six.predict(X_test_six)

mse_lr_six = mean_squared_error(Y_test_six, y_pred_lr_six)
r2_lr_six = r2_score(Y_test_six, y_pred_lr_six)
print(f"Mean Squared Error: {mse_lr_six:.2f}")
print(f"R2 Score: {r2_lr_six:.2f}")

# XGBoost
print("\n--- XGBoost Model ---")
xgb_model_six = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42)
xgb_model_six.fit(X_train_six, Y_train_six)

y_pred_xgb_six = xgb_model_six.predict(X_test_six)

mse_xgb_six = mean_squared_error(Y_test_six, y_pred_xgb_six)
r2_xgb_six = r2_score(Y_test_six, y_pred_xgb_six)
print(f"Mean Squared Error: {mse_xgb_six:.2f}")
print(f"R2 Score: {r2_xgb_six:.2f}")

Take the Next 6-hours Data

--- Linear Regression Model ---
Mean Squared Error: 399.09
R2 Score: 0.27

--- XGBoost Model ---
Mean Squared Error: 302.43
R2 Score: 0.47


In [16]:
# Output the result as a .csv file
results = pd.DataFrame({
    'No': range(len(Y_test_six.flatten())),
    'PM2.5': y_pred_lr_six.flatten()
})
results.to_csv('submission_six.csv', index=False)